Load dependancies

In [64]:
!pip install -e ../gpxpy
from progiter import ProgIter as prog
from strava_export_gpx import main
import pandas as pd
import gpxpy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
locator = RateLimiter(Nominatim(user_agent="strava_mapper").reverse, min_delay_seconds=1)

Obtaining file:///home/watermarkhu/projects/strava_mapper/gpxpy
  Attempting uninstall: gpxpy
    Found existing installation: gpxpy 1.4.2
    Uninstalling gpxpy-1.4.2:
      Successfully uninstalled gpxpy-1.4.2
  Running setup.py develop for gpxpy


Fix TCX and GZ files

In [65]:
folder = "/mnt/d/Users/water_000/Downloads/export_25412727/"
files = main(folder, folder + "activities", folder + "activities.csv")

Reading '/mnt/d/Users/water_000/Downloads/export_25412727/activities.csv'
Found 72 activities 
Unzipping ...
Progress: 
Normalizing ...
activities/2453671243.gpx               | Run          | Morning Run
activities/2459470423.gpx               | Run          | Morning Run
activities/2497042755.gpx               | Snowboard    | Morning Activity
activities/2514112484.gpx               | Run          | Lunch Run
activities/2519401245.gpx               | Run          | Morning Run
activities/2522229121.gpx               | Run          | Morning run
activities/2706238445.gpx               | Run          | Morning Run
activities/2712950931.gpx               | Run          | Lunch Run
activities/2712959895.gpx               | Run          | Morning Run
activities/2717923914.gpx               | Run          | Running
activities/2722525232.gpx               | Run          | Running
activities/2733023399.gpx               | Run          | Morning Run
activities/2733023547.gpx               | R

Save data to Dataframe

In [66]:
data = []
for filename, activity_type in prog(files.items()):
    with open(folder + filename, "r") as file:
        gpxdata = gpxpy.parse(file)

    init_time = gpxdata.tracks[0].segments[0].points[0].time
    for track in gpxdata.tracks:
        locdict = None
        for segment in track.segments:
            for point in segment.points:
                if point.latitude and point.longitude:
                    if not locdict:
                        locdict = locator("{}, {}".format(point.latitude, point.longitude), language='en').raw
                        mydict = dict()
                        for key in ["country", "state", "city"]:
                            mydict[key] = locdict["address"][key] if key in locdict["address"] else "Unknown"
                    data.append(dict(
                        latitude=point.latitude,
                        longitude=point.longitude,
                        time=point.time,
                        type=activity_type,
                        file=filename,
                        **mydict
                    ))

df = pd.DataFrame(data)


 72/72... rate=0.99 Hz, eta=0:00:00, total=0:01:11, wall=18:01 CET


In [70]:
from IPython.display import display
import ipywidgets as widgets
import gmaps
with open("api_key", "r") as file:
    api_key = file.read()
gmaps.configure(api_key=api_key)

class OutletExplorer(object):

    def __init__(self, df, option, max_intensity:int=800, point_radius:int=5):
        """
        Jupyter widget for exploring KFC and Starbucks outlets

        Using checkboxes, the user chooses whether to include
        Starbucks, KFC outlets, both or neither.
        """
        self.max_intensity = max_intensity
        self.point_radius = point_radius
        self._df = df
        self._option = option
        self._options = list(set(self._df.loc[:, option]))
        self._numoptions = len(self._options)
        self._heatmap = None
        self._checkboxes = None
        self._mislider = None
        self._prslider = None

        title_widget = widgets.HTML(
            '<h3>Explore GPX and TCX data</h3>'
        )
        sliders, controls = self._render_controls()
        map_figure = self._render_map()

        self._container = widgets.VBox(
            [title_widget, sliders, controls, map_figure])

    def render(self):
        """ Render the widget """
        display(self._container)

    def _render_map(self):
        """ Render the initial map """
        fig = gmaps.figure()
        self._heatmap = gmaps.heatmap_layer(
            df[['latitude', 'longitude']],
            max_intensity = self.max_intensity,
            point_radius = self.point_radius
        )
        fig.add_layer(self._heatmap)
        self._heatmap.max_intensity = self.max_intensity
        self._heatmap.point_radius = self.point_radius
        return fig

    def _render_controls(self, *args, **kwargs):

        self._mislider = widgets.IntSlider(
            value=self.max_intensity,
            min=0,
            max=2000,
            description='Max Intensity',
            continuous_update=False
        )
        self._mislider.observe(self._on_mi_change, names='value')
        self._prslider = widgets.IntSlider(
            value=self.point_radius,
            min=1,
            max=50,
            description='Point Radius',
            continuous_update=False
        )
        self._prslider.observe(self._on_pr_change, names='value')
        sliders = widgets.VBox([
            self._mislider,
            self._prslider
        ])

        """ Render the checkboxes """
        self._checkboxes = {}
        for option in self._options:
            checkbox = widgets.Checkbox(
                value=True,
                description=option
            )
            checkbox.observe(self._on_checkbox_change, names='value')
            self._checkboxes[option] = checkbox
        controls = widgets.Box(
            list(self._checkboxes.values()),
            layout=widgets.Layout(flex_flow='row wrap')
        )
        return sliders, controls

    def _on_mi_change(self, change):
        self._heatmap.max_intensity = self._mislider.value

    def _on_pr_change(self, change):
        self._heatmap.point_radius = self._prslider.value

    def _on_checkbox_change(self, change):
        """
        Called when the checkboxes change

        This method builds the list of symbols to include on the map,
        based on the current checkbox values. It then updates the
        symbol layer with the new symbol list.
        """

        selected = [option for option, checkbox in self._checkboxes.items() if checkbox.value]
        locations = self._df.loc[df[self._option].isin(selected), ['latitude', 'longitude']]
        self._heatmap.locations = locations
        print(selected, len(locations))
        return self._container

Plot map

In [71]:


GHeatmap(df, "type").render()